### New Version: fixed all bugs from the overwriting previous user input issue. All data is now saved to a folder in GUIproject in a data.csv. Every entry can be encoded now without overwriting.

In [ ]:
import tkinter as tk
from tkinter import simpledialog, scrolledtext
import time
import csv
import os

def save_data(data, filepath):
    headers = ["ID", "Total Characters", "Total Keystrokes", "Total Backspaces", "Time Elapsed", "Input Text 1", "Stress Rating 1"]
    file_exists = os.path.isfile(filepath)
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

def open_summary_window(data, root):
    summary_window = tk.Toplevel(root)
    summary_window.title("Summary of Your Input")
    summary_window.geometry("300x200")
    summary_label = tk.Label(summary_window, text=f"ID: {data['ID']}\n"
                                                  f"Total Characters: {data['Total Characters']}\n"
                                                  f"Total Keystrokes: {data['Total Keystrokes']}\n"
                                                  f"Total Backspaces: {data['Total Backspaces']}\n"
                                                  f"Time Elapsed: {data['Time Elapsed']}", justify="left", wraplength=280)
    summary_label.pack(pady=20)

    return_button = tk.Button(summary_window, text="Return to Beginning", command=lambda: return_to_start(summary_window, root))
    return_button.pack(pady=5)

    close_button = tk.Button(summary_window, text="End and Save", command=lambda: end_and_save(root))
    close_button.pack(pady=5)

def open_stress_window(data, root):
    stress_window = tk.Toplevel(root)
    stress_window.title("Stress Level Rating")
    stress_window.geometry("400x200")
    instruction_text = ("As an employee receiving this message from a supervisor, rate the level of stress "
                        "you experienced while you were drafting a response below. Please only input stress level "
                        "from 0 to 10 (10 as the maximum stress level). Response can include decimals.")
    instruction_label = tk.Label(stress_window, text=instruction_text, wraplength=380)
    instruction_label.pack(pady=20)

    stress_entry = tk.Entry(stress_window)
    stress_entry.pack()

    def save_stress_and_proceed():
        stress_rating = float(stress_entry.get())  # Convert input to float
        data["Stress Rating 1"] = stress_rating
        save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/data.csv")
        stress_window.destroy()
        open_summary_window(data, root)

    submit_button = tk.Button(stress_window, text="Submit", command=save_stress_and_proceed)
    submit_button.pack(pady=20)

def return_to_start(summary_window, root):
    summary_window.destroy()
    open_instructions_window(root)

def end_and_save(root):
    root.quit()
    root.destroy()
    print("All windows closed and program terminated")

def send_response(reply_window, data, root):
    reply_window.destroy()
    open_stress_window(data, root)

def open_reply_window(root, user_id):
    reply_window = tk.Toplevel(root)
    reply_window.title("Reply to Supervisor")
    reply_window.geometry("600x400")

    email_message = ("Dear Team,\n\n"
                     "I hope this message finds you well. As part of our ongoing project, "
                     "I would like you to give me a brief update on your current progress. "
                     "Please include any significant achievements, challenges you are facing, "
                     "and any immediate assistance you might need from the management team.\n\n"
                     "Best regards,\n"
                     "Your Supervisor")
    message_label = tk.Label(reply_window, text=email_message, justify="left", wraplength=580)
    message_label.pack(pady=(10, 20))

    text_box = scrolledtext.ScrolledText(reply_window, width=50, height=10)
    text_box.pack()

    timer_label = tk.Label(reply_window, text="00:00", font=('Helvetica', 14))
    timer_label.pack(anchor='ne')

    start_time = time.time()
    keystrokes = 0
    backspaces = 0
    stop_timer = False

    def update_timer():
        if not stop_timer:
            elapsed_time = time.time() - start_time
            minutes, seconds = divmod(int(elapsed_time), 60)
            timer_label.config(text=f"{minutes:02}:{seconds:02}")
            reply_window.after(1000, update_timer)

    update_timer()

    def key_press(event):
        nonlocal keystrokes, backspaces
        if event.keysym == 'BackSpace':
            backspaces += 1
        keystrokes += 1

    text_box.bind("<KeyPress>", key_press)

    def handle_send():
        nonlocal stop_timer
        stop_timer = True
        characters = len(text_box.get("1.0", "end-1c"))
        input_text = text_box.get("1.0", "end-1c")
        elapsed_time = time.time() - start_time
        total_seconds = int(elapsed_time)  # Save total seconds for CSV
        data = {
            "ID": user_id,
            "Total Characters": characters,
            "Total Keystrokes": keystrokes,
            "Total Backspaces": backspaces,
            "Time Elapsed": total_seconds,
            "Input Text 1": input_text
        }
        reply_window.destroy()
        open_stress_window(data, root)

    send_button = tk.Button(reply_window, text="Send", command=handle_send)
    send_button.pack(pady=10)


def open_instructions_window(root):
    instructions_window = tk.Toplevel(root)
    instructions_window.title("Instructions")
    instructions_window.geometry("600x400")
    instruction_text = ("You have entered your ID. Please read the following instructions carefully:\n"
                        "1. Ensure you are in a quiet environment.\n"
                        "2. Keep your focus on the task.\n"
                        "3. Please do not click SEND until you are satisfied with your response.\n"
                        "4. Please do not click SAVE AND EXIT until your experimenter says so.\n")
    instructions_label = tk.Label(instructions_window, text=instruction_text, justify="left", wraplength=580)
    instructions_label.pack(pady=(20, 10))

    user_id = simpledialog.askstring("Input", "Enter your ID number:", parent=instructions_window)
    if user_id:
        next_button = tk.Button(instructions_window, text="Next", command=lambda: [instructions_window.destroy(), open_reply_window(root, user_id)])
        next_button.pack(pady=10)

def main():
    root = tk.Tk()
    root.withdraw()
    open_instructions_window(root)
    root.mainloop()

if __name__ == "__main__":
    main()


### Next step is editing the messages and then adding a second set of supervisor messages and responses.

In [ ]:
import tkinter as tk
from tkinter import simpledialog, scrolledtext
import time
import csv
import os

def save_data(data, filepath):
    headers = ["ID", "Total Characters1", "Total Keystrokes1", "Total Backspaces1", "Time Elapsed1", "Input Text 1", "Stress Rating 1",
               "Total Characters2", "Total Keystrokes2", "Total Backspaces2", "Time Elapsed2", "Input Text 2", "Stress Rating 2"]
    file_exists = os.path.isfile(filepath)
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

def open_summary_window(data, root, round_number):
    summary_window = tk.Toplevel(root)
    summary_window.title(f"Summary of Your Input - {round_number}")
    summary_window.geometry("300x200")
    index = '1' if round_number == "Round 1" else '2'
    text_display = f"ID: {data['ID']}\nTotal Characters: {data[f'Total Characters{index}']}\nTotal Keystrokes: {data[f'Total Keystrokes{index}']}\nTotal Backspaces: {data[f'Total Backspaces{index}']}\nTime Elapsed: {data[f'Time Elapsed{index}']}"
    summary_label = tk.Label(summary_window, text=text_display, justify="left", wraplength=280)
    summary_label.pack(pady=20)

    if round_number == "Round 2":
        close_button = tk.Button(summary_window, text="Save and Exit", command=lambda: end_and_save(root))
        close_button.pack(pady=5)
        return_button = tk.Button(summary_window, text="Return to Beginning", command=lambda: return_to_start(summary_window, root))
        return_button.pack(pady=5)
    else:
        next_button = tk.Button(summary_window, text="Next", command=lambda: [summary_window.destroy(), open_reply_window(root, data['ID'], "Round 2")])
        next_button.pack(pady=5)

def open_stress_window(data, root, round_number):
    stress_window = tk.Toplevel(root)
    stress_window.title(f"Stress Level Rating - {round_number}")
    stress_window.geometry("400x200")
    instruction_text = ("As an employee receiving this message from a supervisor, rate the level of stress "
                        "you experienced while you were drafting a response below. Please only input stress level "
                        "from 0 to 10 (10 as the maximum stress level). Response can include decimals.")
    instruction_label = tk.Label(stress_window, text=instruction_text, wraplength=380)
    instruction_label.pack(pady=20)

    stress_entry = tk.Entry(stress_window)
    stress_entry.pack()

    def save_stress_and_proceed():
        stress_rating = float(stress_entry.get())  # Convert input to float
        index = '1' if round_number == "Round 1" else '2'
        data[f"Stress Rating {index}"] = stress_rating
        save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/data.csv")
        stress_window.destroy()
        open_summary_window(data, root, round_number)

    submit_button = tk.Button(stress_window, text="Submit", command=save_stress_and_proceed)
    submit_button.pack(pady=20)

def return_to_start(summary_window, root):
    summary_window.destroy()
    open_instructions_window(root)

def end_and_save(root):
    root.quit()
    root.destroy()
    print("All windows closed and program terminated")

def open_reply_window(root, user_id, round_number):
    reply_window = tk.Toplevel(root)
    reply_window.title(f"Reply to Supervisor - {round_number}")
    reply_window.geometry("600x400")

    email_message = ("Dear Team,\n\n"
                     "I hope this message finds you well. As part of our ongoing project, "
                     "I would like you to give me a brief update on your current progress. "
                     "Please include any significant achievements, challenges you are facing, "
                     "and any immediate assistance you might need from the management team.\n\n"
                     "Best regards,\n"
                     "Your Supervisor") if round_number == "Round 2" else (
                    "Dear Team,\n\n"
                    "I hope this message finds you well. As part of our ongoing project, "
                    "I would like you to give me a brief update on your current progress. "
                    "Please include any significant achievements, challenges you are facing, "
                    "and any immediate assistance you might need from the management team.\n\n"
                    "Best regards,\n"
                    "Your Supervisor")

    message_label = tk.Label(reply_window, text=email_message, justify="left", wraplength=580)
    message_label.pack(pady=(10, 20))

    text_box = scrolledtext.ScrolledText(reply_window, width=50, height=10)
    text_box.pack()

    timer_label = tk.Label(reply_window, text="00:00", font=('Helvetica', 14))
    timer_label.pack(anchor='ne')

    start_time = time.time()
    keystrokes = 0
    backspaces = 0
    stop_timer = False

    def update_timer():
        if not stop_timer:
            elapsed_time = time.time() - start_time
            minutes, seconds = divmod(int(elapsed_time), 60)
            timer_label.config(text=f"{minutes:02}:{seconds:02}")
            reply_window.after(1000, update_timer)

    update_timer()

    def key_press(event):
        nonlocal keystrokes, backspaces
        if event.keysym == 'BackSpace':
            backspaces += 1
        keystrokes += 1

    text_box.bind("<KeyPress>", key_press)

    def handle_send():
        nonlocal stop_timer
        stop_timer = True
        characters = len(text_box.get("1.0", "end-1c"))
        input_text = text_box.get("1.0", "end-1c")
        elapsed_time = time.time() - start_time
        total_seconds = int(elapsed_time)  # Save total seconds for CSV
        index = '1' if round_number == "Round 1" else '2'
        data = {
            "ID": user_id,
            "Total Characters" + index: characters,
            "Total Keystrokes" + index: keystrokes,
            "Total Backspaces" + index: backspaces,
            "Time Elapsed" + index: total_seconds,
            "Input Text " + index: input_text
        }
        reply_window.destroy()
        open_stress_window(data, root, round_number)

    send_button = tk.Button(reply_window, text="Send", command=handle_send)
    send_button.pack(pady=10)

def open_instructions_window(root):
    instructions_window = tk.Toplevel(root)
    instructions_window.title("Instructions")
    instructions_window.geometry("600x400")
    instruction_text = ("You have entered your ID. Please read the following instructions carefully:\n"
                        "1. Ensure you are in a quiet environment.\n"
                        "2. Keep your focus on the task.\n"
                        "3. Please do not click SEND until you are satisfied with your response.\n"
                        "4. Please do not click SAVE AND EXIT until your experimenter says so.\n")
    instructions_label = tk.Label(instructions_window, text=instruction_text, justify="left", wraplength=580)
    instructions_label.pack(pady=(20, 10))

    user_id = simpledialog.askstring("Input", "Enter your ID number:", parent=instructions_window)
    if user_id:
        next_button = tk.Button(instructions_window, text="Next", command=lambda: [instructions_window.destroy(), open_reply_window(root, user_id, "Round 1")])
        next_button.pack(pady=10)

def main():
    root = tk.Tk()
    root.withdraw()
    open_instructions_window(root)
    root.mainloop()

if __name__ == "__main__":
    main()

### For the above code, everything was correct except the .csv file, it seems that somethings are a little wrong with the rows. I made some changes below:

In [ ]:
import tkinter as tk
from tkinter import simpledialog, scrolledtext
import time
import csv
import os

def save_data(data, filepath):
    headers = ["ID", "TotalCharacters1", "TotalKeystrokes1", "TotalBackspaces1", "TimeElapsed1", "StressRating1",
               "TotalCharacters2", "TotalKeystrokes2", "TotalBackspaces2", "TimeElapsed2", "StressRating2"]
    file_exists = os.path.isfile(filepath)
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

def open_summary_window(data, root, round_index):
    summary_window = tk.Toplevel(root)
    summary_window.title("Summary of Your Input - Round {round_index}")
    summary_window.geometry("300x200")
    text_display = f"ID: {data['ID']}\n" \
                   f"Total Keystrokes: {data[f'TotalKeystrokes{round_index}']}\n" \
                   f"Total Backspaces: {data[f'TotalBackspaces{round_index}']}\n" \
                   f"Time Elapsed: {data[f'TimeElapsed{round_index}']} seconds\n" \
                   f"Stress Rating: {data[f'StressRating{round_index}']}"
    summary_label = tk.Label(summary_window, text=text_display, justify="left", wraplength=280)
    summary_label.pack(pady=20)
    
    if round_index == "2":
        close_button = tk.Button(summary_window, text="Save and Exit", command=lambda: end_and_save(root))
        close_button.pack(side=tk.LEFT, padx=10, pady=5)
        
        return_button = tk.Button(summary_window, text="Return to Beginning", command=lambda: return_to_start(summary_window, root))
        return_button.pack(side=tk.RIGHT, padx=10, pady=5)
    else:
        next_button = tk.Button(summary_window, text="Next Round", command=lambda: [summary_window.destroy(), open_reply_window(root, data['ID'], "2")])
        next_button.pack(pady=5)

def return_to_start(summary_window, root):
    summary_window.destroy()
    open_instructions_window(root)

def end_and_save(root):
    root.quit()
    root.destroy()
    print("All windows closed and program terminated")
        
def open_stress_window(data, root, round_index):
    stress_window = tk.Toplevel(root)
    stress_window.title("Stress Level Rating")
    stress_window.geometry("400x200")
    instruction_text = "Rate the level of stress experienced (0-10, decimals allowed):"
    instruction_label = tk.Label(stress_window, text=instruction_text, wraplength=380)
    instruction_label.pack(pady=20)
    stress_entry = tk.Entry(stress_window)
    stress_entry.pack(pady=10)
    submit_button = tk.Button(stress_window, text="Submit", command=lambda: save_stress_and_proceed(data, stress_entry.get(), root, stress_window, round_index))
    submit_button.pack(pady=10)

def save_stress_and_proceed(data, stress_rating, root, stress_window, round_index):
    data[f'StressRating{round_index}'] = float(stress_rating)
    stress_window.destroy()
    open_summary_window(data, root, round_index)
    
def open_reply_window(root, user_id, round_index):
    reply_window = tk.Toplevel(root)
    reply_window.title(f"Reply to Supervisor - Round {round_index}")
    reply_window.geometry("600x400")

    email_message = {
        "1": "Dear Team,\n\nI hope this message finds you well. As part of our ongoing project, "
             "I would like you to give me a brief update on your current progress. "
             "Please include any significant achievements, challenges you are facing, "
             "and any immediate assistance you might need from the management team.\n\n"
             "Best regards,\nYour Supervisor",
        "2": "Dear Team,\n\nThank you for your continued effort on the projects. Please provide an update "
             "on the outcomes of the recent strategies and any further challenges that might need "
             "addressing in our next phase.\n\nBest regards,\nYour Supervisor"
    }[round_index]

    message_label = tk.Label(reply_window, text=email_message, justify="left", wraplength=580)
    message_label.pack(pady=(10, 20))

    text_box = scrolledtext.ScrolledText(reply_window, width=50, height=10)
    text_box.pack()

    timer_label = tk.Label(reply_window, text="00:00", font=('Helvetica', 14))
    timer_label.pack(anchor='ne')

    start_time = time.time()
    keystrokes = 0
    backspaces = 0
    stop_timer = False

    def update_timer():
        if not stop_timer:
            elapsed_time = time.time() - start_time
            minutes, seconds = divmod(int(elapsed_time), 60)
            timer_label.config(text=f"{minutes:02}:{seconds:02}")
            reply_window.after(1000, update_timer)

    update_timer()

    def key_press(event):
        nonlocal keystrokes, backspaces
        if event.keysym == 'BackSpace':
            backspaces += 1
        keystrokes += 1

    text_box.bind("<KeyPress>", key_press)

    def handle_send():
        nonlocal stop_timer
        stop_timer = True
        characters = len(text_box.get("1.0", "end-1c"))
        input_text = text_box.get("1.0", "end-1c")
        elapsed_time = time.time() - start_time
        total_seconds = int(elapsed_time)
        data = {
            "ID": user_id,
            f"TotalKeystrokes{round_index}": keystrokes,
            f"TotalBackspaces{round_index}": backspaces,
            f"TimeElapsed{round_index}": total_seconds,
            f"InputText{round_index}": input_text
        }
        reply_window.destroy()
        open_stress_window(data, root, round_index)

    send_button = tk.Button(reply_window, text="Send", command=handle_send)
    send_button.pack(pady=10)


def open_instructions_window(root):
    instructions_window = tk.Toplevel(root)
    instructions_window.title("Instructions")
    instructions_window.geometry("600x400")
    instruction_text = "Enter your ID and follow the instructions to proceed with the task."
    instructions_label = tk.Label(instructions_window, text=instruction_text, justify="left", wraplength=580)
    instructions_label.pack(pady=20)
    user_id = simpledialog.askstring("Input", "Enter your ID number:", parent=instructions_window)
    if user_id:
        instructions_window.destroy()
        open_reply_window(root, user_id, "1")

def main():
    root = tk.Tk()
    root.withdraw()
    open_instructions_window(root)
    root.mainloop()

if __name__ == "__main__":
    main()


### After two hours, I got the above code which partially works, but the stress rating is not in the data put, and also the output csv is offset by a row. I attempt to fix again below

In [ ]:
import tkinter as tk
from tkinter import simpledialog, scrolledtext
import time
import csv
import os

def save_data(data, filepath):
    headers = ["ID", "TotalCharacters1", "TotalKeystrokes1", "TotalBackspaces1", "TimeElapsed1", "InputText1", "StressRating1", 
               "TotalCharacters2", "TotalKeystrokes2", "TotalBackspaces2", "TimeElapsed2", "InputText2", "StressRating2"]
    file_exists = os.path.isfile(filepath)
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

def open_summary_window(data, root, round_index):
    summary_window = tk.Toplevel(root)
    summary_window.title(f"Summary of Your Input - Round {round_index}")
    summary_window.geometry("300x200")
    text_display = (f"ID: {data['ID']}\n"
                    f"Total Keystrokes: {data[f'TotalKeystrokes{round_index}']}\n"
                    f"Total Backspaces: {data[f'TotalBackspaces{round_index}']}\n"
                    f"Time Elapsed: {data[f'TimeElapsed{round_index}']} seconds\n"
                    f"Stress Rating: {data[f'StressRating{round_index}']}")
    summary_label = tk.Label(summary_window, text=text_display, justify="left", wraplength=280)
    summary_label.pack(pady=20)
    
    if round_index == "2":
        close_button = tk.Button(summary_window, text="Save and Exit", command=lambda: end_and_save(root))
        close_button.pack(side=tk.LEFT, padx=10, pady=5)
        return_button = tk.Button(summary_window, text="Return to Beginning", command=lambda: return_to_start(summary_window, root))
        return_button.pack(side=tk.RIGHT, padx=10, pady=5)
    else:
        next_button = tk.Button(summary_window, text="Next Round", command=lambda: [summary_window.destroy(), open_reply_window(root, data['ID'], "2")])
        next_button.pack(pady=5)

def open_reply_window(root, user_id, round_index):
    reply_window = tk.Toplevel(root)
    reply_window.title(f"Reply to Supervisor - Round {round_index}")
    reply_window.geometry("600x400")
    email_message = {
        "1": "大家好,\n\nI hope this message finds you well. As part of our ongoing project, "
             "I would like you to give me a brief update on your current progress. "
             "Please include any significant achievements, challenges you are facing, "
             "and any immediate assistance you might need from the management team.\n\n"
             "Best regards,\nYour Supervisor",
        "2": "Dear Team,\n\nThank you for your continued effort on the projects. Please provide an update "
             "on the outcomes of the recent strategies and any further challenges that might need "
             "addressing in our next phase.\n\nBest regards,\nYour Supervisor"
    }[round_index]
    message_label = tk.Label(reply_window, text=email_message, justify="left", wraplength=580)
    message_label.pack(pady=(10, 20))
    text_box = scrolledtext.ScrolledText(reply_window, width=50, height=10)
    text_box.pack()
    timer_label = tk.Label(reply_window, text="00:00", font=('Helvetica', 14))
    timer_label.pack(anchor='ne')
    start_time = time.time()
    def update_timer():
        elapsed_time = time.time() - start_time
        minutes, seconds = divmod(int(elapsed_time), 60)
        timer_label.config(text=f"{minutes:02}:{seconds:02}")
        if not hasattr(reply_window, 'stop_timer') or not reply_window.stop_timer:
            reply_window.after(1000, update_timer)
    update_timer()
    def handle_send():
        reply_window.stop_timer = True
        characters = len(text_box.get("1.0", "end-1c"))
        input_text = text_box.get("1.0", "end-1c")
        elapsed_time = time.time() - start_time
        data = {
            "ID": user_id,
            f"TotalKeystrokes{round_index}": characters,
            f"TotalBackspaces{round_index}": 0,  # Update if tracking backspaces
            f"TimeElapsed{round_index}": int(elapsed_time),
            f"InputText{round_index}": input_text,
            f"StressRating{round_index}": 0  # Initialized for update in stress rating
        }
        save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")  # Save data after sending
        reply_window.destroy()
        open_stress_window(data, root, round_index)
    send_button = tk.Button(reply_window, text="Send", command=handle_send)
    send_button.pack(pady=10)
    reply_window.protocol("WM_DELETE_WINDOW", handle_send)  # Handle window close as send

def end_and_save(root):
    root.quit()
    root.destroy()
    print("All windows closed and program terminated")

def open_stress_window(data, root, round_index):
    stress_window = tk.Toplevel(root)
    stress_window.title("Stress Level Rating")
    stress_window.geometry("400x200")

    instruction_text = "Rate the level of stress experienced (0-10, decimals allowed):"
    instruction_label = tk.Label(stress_window, text=instruction_text, wraplength=380)
    instruction_label.pack(pady=20)

    stress_entry = tk.Entry(stress_window)
    stress_entry.pack(pady=10)

    submit_button = tk.Button(stress_window, text="Submit", command=lambda: save_stress_and_proceed(data, stress_entry.get(), stress_window, root, round_index))
    submit_button.pack(pady=10)

def save_stress_and_proceed(data, stress_rating, stress_window, root, round_index):
    try:
        stress_rating = float(stress_rating)  # Ensure stress_rating is converted to float
        data[f'StressRating{round_index}'] = stress_rating  # Update the stress rating in data
        stress_window.destroy()  # Close the stress rating window
        save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")  # Save updated data
        open_summary_window(data, root, round_index)  # Open the summary window
    except ValueError:
        # Handle the case where the input is not a valid float
        instruction_label.config(text="Invalid input. Please enter a numeric value (0-10).")
    
def return_to_start(summary_window, root):
    summary_window.destroy()
    open_instructions_window(root)

def open_instructions_window(root):
    instructions_window = tk.Toplevel(root)
    instructions_window.title("Instructions")
    instructions_window.geometry("600x400")
    instruction_text = "Please enter your ID and follow the instructions to proceed with the task."
    instructions_label = tk.Label(instructions_window, text=instruction_text, justify="left", wraplength=580)
    instructions_label.pack(pady=20)
    user_id = simpledialog.askstring("Input", "Enter your ID number:", parent=instructions_window)
    if user_id:
        instructions_window.destroy()
        open_reply_window(root, user_id, "1")

def main():
    root = tk.Tk()
    root.withdraw()
    open_instructions_window(root)
    root.mainloop()

if __name__ == "__main__":
    main()

### Below, I attempted to fix the data output problem. Also, the instruction window was gone.

In [ ]:
import tkinter as tk
from tkinter import simpledialog, scrolledtext
import time
import csv
import os

# Initialize CSV file
def initialize_csv(filepath):
    headers = ["ID", "TotalCharacters1", "TotalKeystrokes1", "TotalBackspaces1", "TimeElapsed1", "InputText1", "StressRating1", 
               "TotalCharacters2", "TotalKeystrokes2", "TotalBackspaces2", "TimeElapsed2", "InputText2", "StressRating2"]
    with open(filepath, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()

# Save data
def save_data(data, filepath):
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=data.keys())
        writer.writerow(data)

# Manage data throughout the session
data_storage = {}


def open_instructions_window(root):
    instructions_window = tk.Toplevel(root)
    instructions_window.title("Instructions")
    instructions_window.geometry("600x400")
    instruction_text = ("You have entered your ID. Please read the following instructions carefully:\n"
                        "1. Ensure you are in a quiet environment.\n"
                        "2. Keep your focus on the task.\n"
                        "3. Please do not click SEND until you are satisfied with your response.\n"
                        "4. Please do not click SAVE AND EXIT until your experimenter says so.\n")
    instructions_label = tk.Label(instructions_window, text=instruction_text, justify="left", wraplength=580)
    instructions_label.pack(pady=(20, 10))
    
    user_id = simpledialog.askstring("Input", "Enter your ID number:", parent=instructions_window)
    if user_id:
        data = {key: '' for key in instructions_label.cget("text").split('\n')}
        data['ID'] = user_id
        instructions_window.destroy()
        open_reply_window(root, user_id, "1", data)

def open_reply_window(root, user_id, round_index, data):
    reply_window = tk.Toplevel(root)
    reply_window.title(f"Reply to Supervisor - Round {round_index}")
    reply_window.geometry("600x400")
    email_message = {
        "1": "大家好,\n\nI hope this message finds you well. As part of our ongoing project, "
             "I would like you to give me a brief update on your current progress. "
             "Please include any significant achievements, challenges you are facing, "
             "and any immediate assistance you might need from the management team.\n\n"
             "Best regards,\nYour Supervisor",
        "2": "Dear Team,\n\nThank you for your continued effort on the projects. Please provide an update "
             "on the outcomes of the recent strategies and any further challenges that might need "
             "addressing in our next phase.\n\nBest regards,\nYour Supervisor"
    }[round_index]
    message_label = tk.Label(reply_window, text=email_message, justify="left", wraplength=580)
    message_label.pack(pady=(10, 20))
    
    text_box = scrolledtext.ScrolledText(reply_window, width=50, height=10)
    text_box.pack()
    timer_label = tk.Label(reply_window, text="00:00", font=('Helvetica', 14))
    timer_label.pack(anchor='ne')
    start_time = time.time()
    
    def update_timer():
        elapsed_time = time.time() - start_time
        minutes, seconds = divmod(int(elapsed_time), 60)
        timer_label.config(text=f"{minutes:02}:{seconds:02}")
        if not hasattr(reply_window, 'stop_timer') or not reply_window.stop_timer:
            reply_window.after(1000, update_timer)
    update_timer()
    
    def handle_send():
        reply_window.stop_timer = True
        characters = len(text_box.get("1.0", "end-1c"))
        input_text = text_box.get("1.0", "end-1c")
        elapsed_time = time.time() - start_time
        data.update({
            f"TotalKeystrokes{round_index}": characters,
            f"TotalBackspaces{round_index}": 0,  # Update if tracking backspaces
            f"TimeElapsed{round_index}": int(elapsed_time),
            f"InputText{round_index}": input_text,
        })
        reply_window.destroy()
        open_stress_window(data, root, round_index)
    send_button = tk.Button(reply_window, text="Send", command=handle_send)
    send_button.pack(pady=10)
    reply_window.protocol("WM_DELETE_WINDOW", handle_send)  # Handle window close as send

def open_stress_window(data, root, round_index):
    stress_window = tk.Toplevel(root)
    stress_window.title("Stress Level Rating")
    stress_window.geometry("400x200")
    
    instruction_text = "Rate the level of stress experienced (0-10, decimals allowed):"
    instruction_label = tk.Label(stress_window, text=instruction_text, wraplength=380)
    instruction_label.pack(pady=20)
    
    stress_entry = tk.Entry(stress_window)
    stress_entry.pack(pady=10)
    
    submit_button = tk.Button(stress_window, text="Submit", command=lambda: save_stress_and_proceed(data, stress_entry.get(), stress_window, root, round_index))
    submit_button.pack(pady=10)

def save_stress_and_proceed(data, stress_rating, stress_window, root, round_index):
    try:
        stress_rating = float(stress_rating)  # Ensure stress_rating is converted to float
        data[f'StressRating{round_index}'] = stress_rating  # Update the stress rating in data
        stress_window.destroy()  # Close the stress rating window
        save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")  # Save updated data
        open_summary_window(data, root, round_index)  # Open the summary window
    except ValueError:
        # Handle the case where the input is not a valid float
        instruction_label.config(text="Invalid input. Please enter a numeric value (0-10).")

def open_summary_window(data, root, round_index):
    summary_window = tk.Toplevel(root)
    summary_window.title(f"Summary of Your Input - Round {round_index}")
    summary_window.geometry("300x200")
    text_display = (f"ID: {data['ID']}\n"
                    f"Total Keystrokes: {data[f'TotalKeystrokes{round_index}']}\n"
                    f"Total Backspaces: {data[f'TotalBackspaces{round_index}']}\n"
                    f"Time Elapsed: {data[f'TimeElapsed{round_index}']} seconds\n"
                    f"Stress Rating: {data[f'StressRating{round_index}']}")
    summary_label = tk.Label(summary_window, text=text_display, justify="left", wraplength=280)
    summary_label.pack(pady=20)
    
    if round_index == "2":
        close_button = tk.Button(summary_window, text="Save and Exit", command=lambda: end_and_save(root))
        close_button.pack(side=tk.LEFT, padx=10, pady=5)
        return_button = tk.Button(summary_window, text="Return to Beginning", command=lambda: return_to_start(summary_window, root))
        return_button.pack(side=tk.RIGHT, padx=10, pady=5)
    else:
        next_button = tk.Button(summary_window, text="Next Round", command=lambda: [summary_window.destroy(), open_reply_window(root, data['ID'], "2", data)])
        next_button.pack(pady=5)

def end_and_save(root):
    root.quit()
    root.destroy()
    print("All windows closed and program terminated")

def return_to_start(summary_window, root):
    summary_window.destroy()
    open_instructions_window(root)

def main():
    root = tk.Tk()
    root.withdraw()
    file_path = "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv"
    initialize_csv(file_path)
    open_instructions_window(root)
    root.mainloop()

if __name__ == "__main__":
    main()


### Program ran fine but output was super weird!

In [ ]:
import tkinter as tk
from tkinter import simpledialog, scrolledtext
import time
import csv
import os

def save_data(data, filepath):
    headers = ["ID", "TotalKeystrokes1", "TotalBackspaces1", "TimeElapsed1", "InputText1", "StressRating1", 
               "TotalKeystrokes2", "TotalBackspaces2", "TimeElapsed2", "InputText2", "StressRating2"]
    file_exists = os.path.isfile(filepath)
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

def open_summary_window(data, root, round_index):
    summary_window = tk.Toplevel(root)
    summary_window.title(f"Summary of Your Input - Round {round_index}")
    summary_window.geometry("300x200")
    text_display = (f"ID: {data['ID']}\n"
                    f"Total Keystrokes: {data[f'TotalKeystrokes{round_index}']}\n"
                    f"Total Backspaces: {data[f'TotalBackspaces{round_index}']}\n"
                    f"Time Elapsed: {data[f'TimeElapsed{round_index}']} seconds\n"
                    f"Stress Rating: {data[f'StressRating{round_index}']}")
    summary_label = tk.Label(summary_window, text=text_display, justify="left", wraplength=280)
    summary_label.pack(pady=20)

    if round_index == "2":
        close_button = tk.Button(summary_window, text="Save and Exit", command=lambda: end_and_save(root))
        close_button.pack(side=tk.LEFT, padx=10, pady=5)
        return_button = tk.Button(summary_window, text="Return to Beginning", command=lambda: return_to_start(summary_window, root))
        return_button.pack(side=tk.RIGHT, padx=10, pady=5)
    else:
        next_button = tk.Button(summary_window, text="Next", command=lambda: [summary_window.destroy(), open_reply_window(root, data['ID'], "2")])
        next_button.pack(pady=5)

def open_reply_window(root, user_id, round_index):
    reply_window = tk.Toplevel(root)
    reply_window.title(f"Reply to Supervisor - Round {round_index}")
    reply_window.geometry("600x400")
    email_message = {
        "1": "Dear Team,\n\nI hope this message finds you well. As part of our ongoing project, "
             "I would like you to give me a brief update on your current progress. "
             "Please include any significant achievements, challenges you are facing, "
             "and any immediate assistance you might need from the management team.\n\n"
             "Best regards,\nYour Supervisor",
        "2": "Dear Team,\n\nThank you for your continued effort on the projects. Please provide an update "
             "on the outcomes of the recent strategies and any further challenges that might need "
             "addressing in our next phase.\n\nBest regards,\nYour Supervisor"
    }[round_index]
    message_label = tk.Label(reply_window, text=email_message, justify="left", wraplength=580)
    message_label.pack(pady=(10, 20))
    text_box = scrolledtext.ScrolledText(reply_window, width=50, height=10)
    text_box.pack()
    timer_label = tk.Label(reply_window, text="00:00", font=('Helvetica', 14))
    timer_label.pack(anchor='ne')
    start_time = time.time()

    def update_timer():
        if not getattr(reply_window, 'stop_timer', False):
            elapsed_time = time.time() - start_time
            minutes, seconds = divmod(int(elapsed_time), 60)
            timer_label.config(text=f"{minutes:02}:{seconds:02}")
            reply_window.after(1000, update_timer)
    update_timer()

    def handle_send():
        reply_window.stop_timer = True
        characters = len(text_box.get("1.0", "end-1c"))
        input_text = text_box.get("1.0", "end-1c")
        elapsed_time = time.time() - start_time
        data = {
            "ID": user_id,
            f"TotalKeystrokes{round_index}": characters,
            f"TotalBackspaces{round_index}": 0,  # Update if tracking backspaces
            f"TimeElapsed{round_index}": int(elapsed_time),
            f"InputText{round_index}": input_text,
            f"StressRating{round_index}": 0  # Initialized for update in stress rating
        }
        save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")  # Save data after sending
        reply_window.destroy()
        open_stress_window(data, root, round_index)
    send_button = tk.Button(reply_window, text="Send", command=handle_send)
    send_button.pack(pady=10)
    reply_window.protocol("WM_DELETE_WINDOW", handle_send)  # Handle window close as send

def end_and_save(root):
    root.quit()
    root.destroy()
    print("All windows closed and program terminated")

def open_stress_window(data, root, round_index):
    stress_window = tk.Toplevel(root)
    stress_window.title("Stress Level Rating")
    stress_window.geometry("400x200")

    instruction_text = "Rate the level of stress experienced (0-10, decimals allowed):"
    instruction_label = tk.Label(stress_window, text=instruction_text, wraplength=380)
    instruction_label.pack(pady=20)

    stress_entry = tk.Entry(stress_window)
    stress_entry.pack(pady=10)

    submit_button = tk.Button(stress_window, text="Submit", command=lambda: save_stress_and_proceed(data, stress_entry.get(), stress_window, root, round_index))
    submit_button.pack(pady=10)

def save_stress_and_proceed(data, stress_rating, stress_window, root, round_index):
    try:
        stress_rating = float(stress_rating)  # Ensure stress_rating is converted to float
        data[f'StressRating{round_index}'] = stress_rating  # Update the stress rating in data
        stress_window.destroy()  # Close the stress rating window
        save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")  # Save updated data
        open_summary_window(data, root, round_index)  # Open the summary window
    except ValueError:
        # Handle the case where the input is not a valid float
        instruction_label.config(text="Invalid input. Please enter a numeric value (0-10).")
    
def return_to_start(summary_window, root):
    summary_window.destroy()
    open_instructions_window(root)

def open_instructions_window(root):
    instructions_window = tk.Toplevel(root)
    instructions_window.title("Instructions")
    instructions_window.geometry("600x400")
    instruction_text = ("You have entered your ID. Please read the following instructions carefully:\n"
                        "1. Ensure you are in a quiet environment.\n"
                        "2. Keep your focus on the task.\n"
                        "3. Please do not click SEND until you are satisfied with your response.\n"
                        "4. Please do not click SAVE AND EXIT until your experimenter says so.\n")
    instructions_label = tk.Label(instructions_window, text=instruction_text, justify="left", wraplength=580)
    instructions_label.pack(pady=20)
    user_id = simpledialog.askstring("Input", "Enter your ID number:", parent=instructions_window)
    if user_id:
        instructions_window.destroy()
        open_reply_window(root, user_id, "1")

def main():
    root = tk.Tk()
    root.withdraw()
    open_instructions_window(root)
    root.mainloop()

if __name__ == "__main__":
    main()


### Problem wasn't fixed above. 

### Below: works well, at least the data is compiled. Seems like they still let me run and record the data but it's all offset and repeated. Every row should only correspond with one user ID.

In [ ]:
import tkinter as tk
from tkinter import simpledialog, scrolledtext
import time
import csv
import os

def save_data(data, filepath):
    headers = ["ID", "TotalKeystrokes1", "TotalBackspaces1", "TimeElapsed1", "InputText1", "StressRating1", 
               "TotalKeystrokes2", "TotalBackspaces2", "TimeElapsed2", "InputText2", "StressRating2"]
    file_exists = os.path.isfile(filepath)
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

def open_summary_window(data, root, round_index):
    summary_window = tk.Toplevel(root)
    summary_window.title(f"Summary of Your Input - Round {round_index}")
    summary_window.geometry("300x200")
    text_display = (f"ID: {data['ID']}\n"
                    f"Total Keystrokes: {data[f'TotalKeystrokes{round_index}']}\n"
                    f"Total Backspaces: {data[f'TotalBackspaces{round_index}']}\n"
                    f"Time Elapsed: {data[f'TimeElapsed{round_index}']} seconds\n"
                    f"Stress Rating: {data[f'StressRating{round_index}']}")
    summary_label = tk.Label(summary_window, text=text_display, justify="left", wraplength=280)
    summary_label.pack(pady=20)

    if round_index == "2":
        close_button = tk.Button(summary_window, text="Save and Exit", command=lambda: end_and_save(root))
        close_button.pack(side=tk.LEFT, padx=10, pady=5)
        return_button = tk.Button(summary_window, text="Return to Beginning", command=lambda: return_to_start(summary_window, root))
        return_button.pack(side=tk.RIGHT, padx=10, pady=5)
    else:
        next_button = tk.Button(summary_window, text="Next", command=lambda: [summary_window.destroy(), open_reply_window(root, data['ID'], "2")])
        next_button.pack(pady=5)

def open_reply_window(root, user_id, round_index):
    reply_window = tk.Toplevel(root)
    reply_window.title(f"Reply to Supervisor - Round {round_index}")
    reply_window.geometry("600x400")
    email_message = {
        "1": "Dear Team,\n\nI hope this message finds you well. As part of our ongoing project, "
             "I would like you to give me a brief update on your current progress. "
             "Please include any significant achievements, challenges you are facing, "
             "and any immediate assistance you might need from the management team.\n\n"
             "Best regards,\nYour Supervisor",
        "2": "Dear Team,\n\nThank you for your continued effort on the projects. Please provide an update "
             "on the outcomes of the recent strategies and any further challenges that might need "
             "addressing in our next phase.\n\nBest regards,\nYour Supervisor"
    }[round_index]
    message_label = tk.Label(reply_window, text=email_message, justify="left", wraplength=580)
    message_label.pack(pady=(10, 20))
    text_box = scrolledtext.ScrolledText(reply_window, width=50, height=10)
    text_box.pack()
    timer_label = tk.Label(reply_window, text="00:00", font=('Helvetica', 14))
    timer_label.pack(anchor='ne')
    start_time = time.time()

    def update_timer():
        if not getattr(reply_window, 'stop_timer', False):
            elapsed_time = time.time() - start_time
            minutes, seconds = divmod(int(elapsed_time), 60)
            timer_label.config(text=f"{minutes:02}:{seconds:02}")
            reply_window.after(1000, update_timer)
    update_timer()

    def handle_send():
        reply_window.stop_timer = True
        characters = len(text_box.get("1.0", "end-1c"))
        input_text = text_box.get("1.0", "end-1c")
        elapsed_time = time.time() - start_time
        data = {
            "ID": user_id,
            f"TotalKeystrokes{round_index}": characters,
            f"TotalBackspaces{round_index}": 0,  # Update if tracking backspaces
            f"TimeElapsed{round_index}": int(elapsed_time),
            f"InputText{round_index}": input_text,
            f"StressRating{round_index}": 0  # Initialized for update in stress rating
        }
        save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")  # Save data after sending
        reply_window.destroy()
        open_stress_window(data, root, round_index)
    send_button = tk.Button(reply_window, text="Send", command=handle_send)
    send_button.pack(pady=10)
    reply_window.protocol("WM_DELETE_WINDOW", handle_send)  # Handle window close as send

def end_and_save(root):
    root.quit()
    root.destroy()
    print("All windows closed and program terminated")

def open_stress_window(data, root, round_index):
    stress_window = tk.Toplevel(root)
    stress_window.title("Stress Level Rating")
    stress_window.geometry("400x200")

    instruction_text = "Rate the level of stress experienced (0-10, decimals allowed):"
    instruction_label = tk.Label(stress_window, text=instruction_text, wraplength=380)
    instruction_label.pack(pady=20)

    stress_entry = tk.Entry(stress_window)
    stress_entry.pack(pady=10)

    submit_button = tk.Button(stress_window, text="Submit", command=lambda: save_stress_and_proceed(data, stress_entry.get(), stress_window, root, round_index))
    submit_button.pack(pady=10)

def save_stress_and_proceed(data, stress_rating, stress_window, root, round_index):
    try:
        stress_rating = float(stress_rating)  # Ensure stress_rating is converted to float
        data[f'StressRating{round_index}'] = stress_rating  # Update the stress rating in data
        stress_window.destroy()  # Close the stress rating window
        save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")  # Save updated data
        open_summary_window(data, root, round_index)  # Open the summary window
    except ValueError:
        # Handle the case where the input is not a valid float
        instruction_label.config(text="Invalid input. Please enter a numeric value (0-10).")
    
def return_to_start(summary_window, root):
    summary_window.destroy()
    open_instructions_window(root)

def open_instructions_window(root):
    instructions_window = tk.Toplevel(root)
    instructions_window.title("Instructions")
    instructions_window.geometry("600x400")
    instruction_text = ("You have entered your ID. Please read the following instructions carefully:\n"
                        "1. Ensure you are in a quiet environment.\n"
                        "2. Keep your focus on the task.\n"
                        "3. Please do not click SEND until you are satisfied with your response.\n"
                        "4. Please do not click SAVE AND EXIT until your experimenter says so.\n")
    instructions_label = tk.Label(instructions_window, text=instruction_text, justify="left", wraplength=580)
    instructions_label.pack(pady=20)
    user_id = simpledialog.askstring("Input", "Enter your ID number:", parent=instructions_window)
    if user_id:
        instructions_window.destroy()
        open_reply_window(root, user_id, "1")

def main():
    root = tk.Tk()
    root.withdraw()
    open_instructions_window(root)
    root.mainloop()

if __name__ == "__main__":
    main()


### Because the issue persists, I want to change a different method. Instead of all saving into one csv, I want the round 1 to be saved into GUIdata1.csv inthe same file path and the data from round 2 to be saved into GUIdata2.csv. Every user ID should still only be in one row and one row only. Ultimately I don't think this was successful.

In [ ]:
import tkinter as tk
from tkinter import simpledialog, scrolledtext
import time
import csv
import os

def save_data(data, filepath):
    headers = list(data.keys())  # Dynamically create headers based on data keys
    file_exists = os.path.isfile(filepath)
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

def open_reply_window(root, user_id, round_index):
    reply_window = tk.Toplevel(root)
    reply_window.title(f"Reply to Supervisor - Round {round_index}")
    reply_window.geometry("600x400")
    email_message = {
        "1": "Dear Team,\n\nI hope this message finds you well. As part of our ongoing project, "
             "I would like you to give me a brief update on your current progress. "
             "Please include any significant achievements, challenges you are facing, "
             "and any immediate assistance you might need from the management team.\n\n"
             "Best regards,\nYour Supervisor",
        "2": "Dear Team,\n\nThank you for your continued effort on the projects. Please provide an update "
             "on the outcomes of the recent strategies and any further challenges that might need "
             "addressing in our next phase.\n\nBest regards,\nYour Supervisor"
    }[round_index]
    message_label = tk.Label(reply_window, text=email_message, justify="left", wraplength=580)
    message_label.pack(pady=(10, 20))
    text_box = scrolledtext.ScrolledText(reply_window, width=50, height=10)
    text_box.pack()
    timer_label = tk.Label(reply_window, text="00:00", font=('Helvetica', 14))
    timer_label.pack(anchor='ne')
    start_time = time.time()

    def update_timer():
        if not getattr(reply_window, 'stop_timer', False):
            elapsed_time = time.time() - start_time
            minutes, seconds = divmod(int(elapsed_time), 60)
            timer_label.config(text=f"{minutes:02}:{seconds:02}")
            reply_window.after(1000, update_timer)
    update_timer()

    def handle_send():
        reply_window.stop_timer = True
        characters = len(text_box.get("1.0", "end-1c"))
        input_text = text_box.get("1.0", "end-1c")
        elapsed_time = time.time() - start_time
        data = {
            "ID": user_id,
            f"TotalKeystrokes{round_index}": characters,
            f"TotalBackspaces{round_index}": 0,
            f"TimeElapsed{round_index}": int(elapsed_time),
            f"InputText{round_index}": input_text,
            f"StressRating{round_index}": 0  # This will be updated in the stress window
        }
        reply_window.destroy()
        open_stress_window(data, root, round_index)
    send_button = tk.Button(reply_window, text="Send", command=handle_send)
    send_button.pack(pady=10)
    reply_window.protocol("WM_DELETE_WINDOW", handle_send)

def open_stress_window(data, root, round_index):
    stress_window = tk.Toplevel(root)
    stress_window.title("Stress Level Rating")
    stress_window.geometry("400x200")
    instruction_text = "Rate the level of stress experienced (0-10, decimals allowed):"
    instruction_label = tk.Label(stress_window, text=instruction_text, wraplength=380)
    instruction_label.pack(pady=20)
    stress_entry = tk.Entry(stress_window)
    stress_entry.pack(pady=10)

    def save_stress_and_proceed(): #now a part of open stress window?
        try:
            stress_rating = float(stress_entry.get())
            data[f'StressRating{round_index}'] = stress_rating
            filepath = f"/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata{round_index}.csv"
            save_data(data, filepath)
            stress_window.destroy()
            if round_index == "1":
                open_summary_window(data, root, "2")
            else:
                end_and_save(root)
        except ValueError:
            instruction_label.config(text="Invalid input. Please enter a numeric value (0-10).")

    submit_button = tk.Button(stress_window, text="Submit", command=save_stress_and_proceed)
    submit_button.pack(pady=10)

def open_summary_window(data, root, round_index):
    summary_window = tk.Toplevel(root)
    summary_window.title(f"Summary of Your Input - Round {round_index}")
    summary_window.geometry("300x200")
    # Prepare the display text by dynamically listing all relevant data entries
    text_display = ""
    for key in sorted(data.keys()):
        if key.endswith(round_index) or key in ['ID', 'StressRating']:
            text_display += f"{key}: {data[key]}\n"
    
    summary_label = tk.Label(summary_window, text=text_display, justify="left", wraplength=280)
    summary_label.pack(pady=20)
    
    if round_index == "2":
        close_button = tk.Button(summary_window, text="Save and Exit", command=lambda: end_and_save(root))
        close_button.pack(side=tk.LEFT, padx=10, pady=5)
        return_button = tk.Button(summary_window, text="Return to Beginning", command=lambda: return_to_start(summary_window, root))
        return_button.pack(side=tk.RIGHT, padx=10, pady=5)
    else:
        next_button = tk.Button(summary_window, text="Next", command=lambda: [summary_window.destroy(), open_reply_window(root, data['ID'], "2")])
        next_button.pack(pady=5)

def end_and_save(root):
    print("All windows closed and program terminated")
    root.quit()
    root.destroy()

def return_to_start(summary_window, root):
    summary_window.destroy()
    open_instructions_window(root)

def open_instructions_window(root):
    instructions_window = tk.Toplevel(root)
    instructions_window.title("Instructions")
    instructions_window.geometry("600x400")
    instruction_text = ("You have entered your ID. Please read the following instructions carefully:\n"
                        "1. Ensure you are in a quiet environment.\n"
                        "2. Keep your focus on the task.\n"
                        "3. Please do not click SEND until you are satisfied with your response.\n"
                        "4. Please do not click SAVE AND EXIT until your experimenter says so.\n")
    instructions_label = tk.Label(instructions_window, text=instruction_text, justify="left", wraplength=580)
    instructions_label.pack(pady=20)
    user_id = simpledialog.askstring("Input", "Enter your ID number:", parent=instructions_window)
    if user_id:
        instructions_window.destroy()
        open_reply_window(root, user_id, "1")

def main():
    root = tk.Tk()
    root.withdraw()
    open_instructions_window(root)
    root.mainloop()

if __name__ == "__main__":
    main()


The above code doesn't work. The summary window is incomplete.

### Separate (debugged the below code in Spyder). IT WORKED

In [1]:
import tkinter as tk
from tkinter import simpledialog, scrolledtext
import time
import csv
import os

def save_data(data, filepath):
    headers = ["ID", "TotalKeystrokes1", "TotalBackspaces1", "TimeElapsed1", "InputText1", "StressRating1", 
               "TotalKeystrokes2", "TotalBackspaces2", "TimeElapsed2", "InputText2", "StressRating2"]
    file_exists = os.path.isfile(filepath)
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

def open_summary_window(data, root, round_index):
    summary_window = tk.Toplevel(root)
    summary_window.title(f"Summary of Your Input - Round {round_index}")
    summary_window.geometry("300x200")
    text_display = (f"ID: {data['ID']}\n"
                    f"Total Keystrokes: {data[f'TotalKeystrokes{round_index}']}\n"
                    f"Total Backspaces: {data[f'TotalBackspaces{round_index}']}\n"
                    f"Time Elapsed: {data[f'TimeElapsed{round_index}']} seconds\n"
                    f"Stress Rating: {data[f'StressRating{round_index}']}")
    summary_label = tk.Label(summary_window, text=text_display, justify="left", wraplength=280)
    summary_label.pack(pady=20)

    if round_index == "2":
        save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")
        close_button = tk.Button(summary_window, text="Save and Exit", command=lambda: end_and_save(root))
        close_button.pack(side=tk.LEFT, padx=10, pady=5)
        return_button = tk.Button(summary_window, text="Return to Beginning", command=lambda: return_to_start(summary_window, root))
        return_button.pack(side=tk.RIGHT, padx=10, pady=5)
    else:
        next_button = tk.Button(summary_window, text="Next", command=lambda: [summary_window.destroy(), open_reply_window(root, data["ID"], "2", data)])
        next_button.pack(pady=5)

def open_reply_window(root, user_id, round_index, data):
    reply_window = tk.Toplevel(root)
    reply_window.title(f"Reply to Supervisor - Round {round_index}")
    reply_window.geometry("600x400")
    email_message = {
        "1": "你好,\n\n我注意到你之前请假，现在可能需要时间来重新调整状态。 "
             "然而，我也想提醒你，我们的项目进展需要你的及时参与和努力，所以需要你加紧完成。 "
             "就目前来看，你之前请假的理由不足以解释你现在仍未完成项目的情况。 "
             "我希望你能赶紧进入状态投入工作，确保我们组能够按时完成项目。\n\n"
             "有问题请及时与我沟通。\n--项目组组长",
        "2": "Hello,\n\nI noticed that you previously took leave and may now need time to readjust."
             "However, I also want to remind you that the progress of our project requires your timely"
             "participation and effort, so we need you to expedite your progress."
             "Based on the current situation, the reasons for your previous leave are insufficient"
             "to explain why you have not yet completed the project."
             "Please communicate with me promptly if you have any questions."
             "\n\nBest regards,\nYour Supervisor"
    }[round_index]
    message_label = tk.Label(reply_window, text=email_message, justify="left", wraplength=580)
    message_label.pack(pady=(10, 20))
    text_box = scrolledtext.ScrolledText(reply_window, width=50, height=10)
    text_box.pack()
    timer_label = tk.Label(reply_window, text="00:00", font=('Helvetica', 14))
    timer_label.pack(anchor='ne')
    start_time = time.time()

    def update_timer():
        if not getattr(reply_window, 'stop_timer', False):
            elapsed_time = time.time() - start_time
            minutes, seconds = divmod(int(elapsed_time), 60)
            timer_label.config(text=f"{minutes:02}:{seconds:02}")
            reply_window.after(1000, update_timer)
    update_timer()

    def handle_send(data):
        reply_window.stop_timer = True
        characters = len(text_box.get("1.0", "end-1c"))
        input_text = text_box.get("1.0", "end-1c")
        elapsed_time = time.time() - start_time
        data[f"TotalKeystrokes{round_index}"] = characters
        data[f"TotalBackspaces{round_index}"] = 0 # Update if tracking backspaces
        data[f"TimeElapsed{round_index}"]     = int(elapsed_time)
        data[f"InputText{round_index}"]       = input_text
        data[f"StressRating{round_index}"]    = 0 # Initialized for update in stress rating
        #save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")  # Save data after sending
        reply_window.destroy()
        open_stress_window(data, root, round_index)
    send_button = tk.Button(reply_window, text="Send", command=lambda: handle_send(data))
    send_button.pack(pady=10)
    reply_window.protocol("WM_DELETE_WINDOW", lambda: handle_send(data))  # Handle window close as send

def end_and_save(root):
    root.quit()
    root.destroy()
    print("All windows closed and program terminated")

def open_stress_window(data, root, round_index):
    stress_window = tk.Toplevel(root)
    stress_window.title("Stress Level Rating")
    stress_window.geometry("400x200")

    instruction_text = "Rate the level of stress experienced (0-10, decimals allowed):"
    instruction_label = tk.Label(stress_window, text=instruction_text, wraplength=380)
    instruction_label.pack(pady=20)

    stress_entry = tk.Entry(stress_window)
    stress_entry.pack(pady=10)

    submit_button = tk.Button(stress_window, text="Submit", command=lambda: save_stress_and_proceed(data, stress_entry.get(), stress_window, root, round_index))
    submit_button.pack(pady=10)

def save_stress_and_proceed(data, stress_rating, stress_window, root, round_index):
    try:
        stress_rating = float(stress_rating)  # Ensure stress_rating is converted to float
        data[f'StressRating{round_index}'] = stress_rating  # Update the stress rating in data
        stress_window.destroy()  # Close the stress rating window
        #save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")  # Save updated data
        open_summary_window(data, root, round_index)  # Open the summary window
    except ValueError:
        # Handle the case where the input is not a valid float
        instruction_label.config(text="Invalid input. Please enter a numeric value (0-10).")
    
def return_to_start(summary_window, root):
    summary_window.destroy()
    open_instructions_window(root)

def open_instructions_window(root):
    instructions_window = tk.Toplevel(root)
    instructions_window.title("Instructions")
    instructions_window.geometry("600x400")
    instruction_text = ("You have entered your ID. Please read the following instructions carefully:\n"
                        "1. You will read two emails sent by your supervisor.\n"
                        "2. Round 1 will be in Chinese and Round 2 in English.\n"
                        "3. Please respond ONLY in ENGLISH.\n"
                        "4. Please do not click SEND until you are satisfied with your response.\n"
                        "5. Please do not click SAVE AND EXIT until your experimenter says so.\n")
    instructions_label = tk.Label(instructions_window, text=instruction_text, justify="left", wraplength=580)
    instructions_label.pack(pady=20)
    user_id = simpledialog.askstring("Input", "Enter your ID number:", parent=instructions_window)
    if user_id:
        instructions_window.destroy()
        data = {}
        data.setdefault("ID", user_id) 
        open_reply_window(root, user_id, "1", data)

def main():
    root = tk.Tk()
    root.withdraw()
    open_instructions_window(root)
    root.mainloop()

if __name__ == "__main__":
    main()

All windows closed and program terminated
